In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/reviews.csv', index_col='filename')

In [ ]:
df.head(10)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.5)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
text_clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,1))),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
    ])

In [ ]:
text_type = 'lemmatized_text'

In [ ]:
text_clf.fit(df_train[text_type], df_train['review'])

In [ ]:
text_clf['vect'].get_feature_names()[100:110]

In [ ]:
len(text_clf['vect'].get_feature_names())

In [ ]:
text_clf['tfidf'].idf_

In [ ]:
text_clf['clf']

In [ ]:
text_clf['clf'].feature_importances_

In [ ]:
feature_df = pd.DataFrame({"feature": text_clf['vect'].get_feature_names(), 
                           'importance': text_clf['clf'].feature_importances_})

In [ ]:
features_sorted = feature_df.sort_values(by=['importance'], ascending=False)

In [ ]:
pd.set_option("max_rows", None)
features_sorted.head(100)

In [ ]:
# you can do this to see the word distribution vector, but with this much data, you might not want to...
#X_train = text_clf['vect'].fit_transform(df['alpha_text'])
#print(X_train[:100])

In [ ]:
len(features_sorted)

In [ ]:
y_pred = text_clf.predict(df_test[text_type])

In [ ]:
#y_pred

In [ ]:
df_test['pred'] = y_pred

In [ ]:
y_proba = text_clf.predict_proba(df_test[text_type])

In [ ]:
y_proba

In [ ]:
pos_proba = [item[1] for item in y_proba]

In [ ]:
#pos_proba

In [ ]:
df_test['proba'] = pos_proba

In [ ]:
df_test.head(10)

In [ ]:
# try a few lines of cleaned text - what would be retained from these sentences? 
document_text = """its not especially good but it isnt especially bad either"""
#document_text = """movie was bad acting was poor script was terrible plot was trite"""
#document_text = """movie was great acting was excellent script was magnificent and wonderful plot was exciting"""

In [ ]:
# let's try: https://www.rogerebert.com/reviews/marmaduke-2010
# what would be retained from this from our data cleaning pipeline? Does it make a big difference?

In [ ]:
text_clf.predict_proba([document_text])

Accuracy:
How often did the predicted value match the actual value

Precision:
When you predicted a positive result, how often was the actual value positive?

Recall:
When the actual value was positive, how often did you predict a positive value?

Visual:
https://en.wikipedia.org/wiki/Precision_and_recall

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve

In [ ]:
precision, recall, fscore, train_support = precision_recall_fscore_support(df_test['review'], y_pred, pos_label=1, average='binary')
print('Precision: {} / Recall: {} / Accuracy: {} / fscore: {}'.format(
    round(precision, 3), round(recall, 3), (round((y_pred==df_test['review']).sum()/len(y_pred), 3)), round(fscore, 3)))

In [ ]:
# my results for multiple runs
# body_text - Precision: 0.735 / Recall: 0.742 / Accuracy: 0.745
# alpha_text - Precision: 0.788 / Recall: 0.65 / Accuracy: 0.74
# nostop_text - Precision: 0.799 / Recall: 0.704 / Accuracy: 0.76
# lemmatized_text - 0.789 / Recall: 0.65 / Accuracy: 0.734

In [ ]:
# cross validation
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

from sklearn.model_selection import cross_val_score

In [ ]:
cross_val = cross_val_score(text_clf, df_test['lemmatized_text'], y_pred, cv=5)

In [ ]:
cross_val